In [1]:
import pandas as pd
import numpy as np 

In [2]:
df = pd.read_csv('/Users/dynaneshwarijangale/Downloads/Tennis.csv')
print(df)

   Attributes     Groups                        Values
0     BallPos    Serving  Right Section of Middle Road
1         NaN        NaN                    Right Zone
2         NaN        NaN   Left Section of Middle Road
3         NaN        NaN                     Left Zone
4         NaN        NaN  Outside Corner of Right Zone
..        ...        ...                           ...
59       Spin    Up spin                      Top Spin
60        NaN    No spin                      Flat Hit
61        NaN  Down spin                     Down Spin
62        NaN     Others                    Data Error
63        NaN        NaN                         Score

[64 rows x 3 columns]


In [6]:
dataset = [
    {"BallPos": "Serving", "StrikePos": "Serving", "StrikeTech": "Serving", "Spin": "Up spin"},
    {"BallPos": "Right Zone", "StrikePos": "Right Zone", "StrikeTech": "Overhand Serving", "Spin": "Top Spin"},
    {"BallPos": "Left Zone", "StrikePos": "Left Zone", "StrikeTech": "Underhand Serving", "Spin": "Down Spin"},
]

labels = [0, 1, 1]  

num_trees = 3  

one_hot_encoded_data = []
for entry in dataset:
    encoded_entry = []
    for attr, values in entry.items():
        if attr == "BallPos":
            encoded_entry.extend([1 if values == pos else 0 for pos in ["Serving", "Right Zone", "Left Zone"]])
        elif attr == "StrikePos":
            encoded_entry.extend([1 if values == pos else 0 for pos in ["Serving", "Right Zone", "Left Zone"]])
        elif attr == "StrikeTech":
            encoded_entry.extend([1 if values == tech else 0 for tech in ["Serving", "Overhand Serving", "Underhand Serving"]])
        elif attr == "Spin":
            encoded_entry.extend([1 if values == spin else 0 for spin in ["Up spin", "Top Spin", "Down Spin"]])
    one_hot_encoded_data.append(encoded_entry)

def predict(tree, data_point):
    while True:
        if "value" in tree:
            return tree["value"]
        attr, threshold, left, right = tree["attr"], tree["threshold"], tree["left"], tree["right"]
        if data_point[attr] < threshold:
            tree = left
        else:
            tree = right

def build_tree(data, labels):
    return {"attr": None, "threshold": None, "left": None, "right": None, "value": sum(labels) / len(labels)}

def gradient_boosting_predict(trees, data_point):
    return sum(predict(tree, data_point) for tree in trees) / len(trees)

trees = []
for _ in range(num_trees):
    tree = build_tree(one_hot_encoded_data, labels)
    trees.append(tree)

predictions = [gradient_boosting_predict(trees, data_point) for data_point in one_hot_encoded_data]

true_positives, true_negatives, false_positives, false_negatives = 0, 0, 0, 0
for prediction, label in zip(predictions, labels):
    if prediction >= 0.5 and label == 1:
        true_positives += 1
    elif prediction >= 0.5 and label == 0:
        false_positives += 1
    elif prediction < 0.5 and label == 1:
        false_negatives += 1
    elif prediction < 0.5 and label == 0:
        true_negatives += 1

accuracy = (true_positives + true_negatives) / len(labels)
precision = true_positives / (true_positives + false_positives) if (true_positives + false_positives) != 0 else 0
recall = true_positives / (true_positives + false_negatives) if (true_positives + false_negatives) != 0 else 0
f1_score = 2 * precision * recall / (precision + recall) if (precision + recall) != 0 else 0

confusion_matrix = [[true_negatives, false_positives], [false_negatives, true_positives]]

print("Accuracy:", accuracy)
print("Precision:", precision)
print("Recall:", recall)
print("F1 Score:", f1_score)
print("Confusion Matrix:")
for row in confusion_matrix:
    print(row)


Accuracy: 0.6666666666666666
Precision: 0.6666666666666666
Recall: 1.0
F1 Score: 0.8
Confusion Matrix:
[0, 1]
[0, 2]
